In [2]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import random
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from apriltags_ros.msg import AprilTagDetection, AprilTagDetectionArray
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori
from scoop.msg import stone_pose_ver
from sensor_msgs.msg import Image, CameraInfo
import message_filters

from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point

import urx
import math3d as m3d
import logging

import matplotlib.pyplot as plt

rospy.init_node('scoop', anonymous=True) #initialize the node



In [3]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1607067242.891848]: Client test: Starting sending goals


In [4]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
servo_pub = rospy.Publisher('servo1', UInt16, queue_size=10)

In [5]:
servo2_pub = rospy.Publisher('servo2', UInt16, queue_size=10)

def softtip():
    servo2_pub.publish(60) #bringing soft-pad at the tip
    rospy.sleep(1.8)
    servo2_pub.publish(90)
    
def hardtip():
    servo2_pub.publish(120) #bringing soft-pad at the tip
    rospy.sleep(1.8)
    servo2_pub.publish(90)

In [6]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.3, 0.8)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.3, 0.8)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.3, 0.8)
    
def move_ref_base(target_ref_transform, ref_frame, acc=0.1, vel=0.02, wait=True):
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.007
    world2tcp.pos.y-=0.001
    frame_to_tcp = m3d.Transform.get_inverse(world2tcp.inverse * m3d.Transform(ref_frame))
    temp1 = m3d.Transform(frame_to_tcp)
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = frame_to_tcp.orient.inverse * delta_translation
    rob.add_pose_tool(Delta_tcp, acc, vel, wait=True, command="movel", threshold=None)
    
def move_ref_tcp(target_ref_transform, tcp_to_frame, acc=0.1, vel=0.02, wait=True):  
    temp1 =  m3d.Transform.get_inverse(m3d.Transform(tcp_to_frame))
    #print temp1
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = temp1.orient.inverse * delta_translation
    #print m3d.Transform(Delta_tcp)
    rob.add_pose_tool(Delta_tcp, acc, vel, wait=True, command="movel", threshold=None)

In [7]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
    worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
    return [worldTstone[0]+0.01634+0.00074+0.0015, worldTstone[1]+0.02429, worldTstone[2]]

In [8]:
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, Go_stone_dir_sorted, yaw_pitch_normal_set
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z
        yaw = msg.yaw
        pitch = msg.pitch
        normal = msg.normal
        dir_depth_inner = msg.dir_depth_inner
        dir_depth_outer = msg.dir_depth_outer
        dir_depth_outer_minus_inner = []
        for i in range(len(dir_depth_inner)):
            dir_depth_outer_minus_inner.append(dir_depth_outer[i]-dir_depth_inner[i])
        dir_depth_outer_minus_inner_argsort = np.argsort(-np.array(dir_depth_outer_minus_inner)).tolist()
        dir_depth_outer_minus_inner_argsort_copy = dir_depth_outer_minus_inner_argsort[:]
        for index in dir_depth_outer_minus_inner_argsort:
            if dir_depth_outer[index]-dir_depth_inner[index]<0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
            elif dir_depth_outer[index]==0 or dir_depth_inner[index]==0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
        dir_depth_outer_minus_inner_argsort=dir_depth_outer_minus_inner_argsort_copy[:]
        hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        Go_stone_dir_sorted.append(dir_depth_outer_minus_inner_argsort)
        yaw_pitch_normal_set.append([yaw,pitch,normal])
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori, hori_stone_pose_callback)

In [9]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, gripper_direction_set, gripper_aperture_set
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        gripper_direction_set = []
        gripper_aperture_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z 
        box = json.loads(msg.box)
        ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
        #print box
        length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
        length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
        if length1 > length2:
            try:
                if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                gripper_direction_set.append(temp_angle)
            except:
                gripper_direction_set.append(0)
            gripper_aperture_set.append(length2+0.004)
        else:
            try:
                if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                gripper_direction_set.append(temp_angle)
            except:
                gripper_direction_set.append(0)
            gripper_aperture_set.append(length1+0.004)        
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver, ver_stone_pose_callback)

In [10]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [11]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [12]:
def poke_grasp_execute(Go_index):
    global hori_pose_Go_stone_set, yaw_pitch_normal_set
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = 0.002
    tcp2fingertip.pos.y = 0  
    tcp2fingertip.pos.z = 0.354
    fingertip2tcp = np.linalg.inv(tcp2fingertip.get_matrix())
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    normal=yaw_pitch_normal_set[Go_index][2]
    n_normalize = normal / np.linalg.norm(normal)
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    fingertip2normal = np.matmul(np.matmul(fingertip2tcp, tcp2cam), n_normalize).tolist()[0]
    alpha = atan2(fingertip2normal[1], fingertip2normal[0])
    if str(alpha) == 'nan':
        print yaw_pitch_normal_set
        return False
    beta = -(pi/2+atan2(fingertip2normal[2], sqrt(fingertip2normal[0]**2+fingertip2normal[1]**2)))
    if str(beta) == 'nan':
        return False
    if abs(beta)>(35*pi/180.0):
        return False
    hardtip()
    go_to_safe_place()
    servo_pub.publish(100)
    rob.movel_tool((0,0,0,0,0,alpha), acc=0.1, vel=0.2, wait=True)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.008
    world2tcp.pos.y-=0.001
    world2tcp.orient=np.array([[0,1,0],[1,0,0],[0,0,-1]])
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hori_pose_Go_stone_set[Go_index][0]-world2fingertip.pos.x,hori_pose_Go_stone_set[Go_index][1]-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.008
    world2tcp.pos.y-=0.001
    world2tcp.orient=np.array([[0,1,0],[1,0,0],[0,0,-1]])
    world2fingertip=(world2tcp*tcp2fingertip)
    fingertip2stone=np.transpose(np.matmul(np.linalg.inv(world2fingertip.get_matrix()), np.transpose(np.array([hori_pose_Go_stone_set[Go_index]+[1]]))))[0].tolist()[0][0:-1]
    rob.set_tcp((0.002, 0, 0.354, 0, 0, 0))
    rospy.sleep(0.5)
    #if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
    rob.movel_tool((0,0,0,0,beta,0), acc=0.3, vel=0.8, wait=True)
    #rob.translate_tool((0.004+0.0015*int(abs(beta)>15)+0.0025*int(abs(beta)>20), 0, 0), acc=0.06, vel=0.08)
    rob.translate_tool((0.003, 0, 0), acc=0.1, vel=0.1)
    psi = radians(27)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    rob.translate((0, 0, -(fingertip2stone[2]-0.010)), acc=0.1, vel=0.1, wait=True)
    rob.translate_tool((0, 0, 0.0265), acc=0.01, vel=0.05)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.2, force=1, block=False)
    rospy.sleep(1)
    rob.translate_tool((0, 0, -0.08), acc=0.1, vel=0.2)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    return True

In [13]:
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import LinearRing

def scoop_collision_check(Go_index):
    global hori_pose_Go_stone_set, Go_stone_dir_sorted, hard_fingertip_position, gripper_orientation
    for ori_index in Go_stone_dir_sorted[Go_index]:
        hard_fingertip_position = [hori_pose_Go_stone_set[Go_index][0]+0.01*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-0.01*cos(ori_index*45*pi/180)]
        gripper_orientation = ori_index*45
        hard_fingertip_position1 = [hard_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)]
        hard_fingertip_position2 = [hard_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]
        if LineString([hard_fingertip_position1, hard_fingertip_position, hard_fingertip_position2]).within(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.005, cap_style=2, join_style=2))==False:
            continue
        soft_fingertip_position = [hori_pose_Go_stone_set[Go_index][0]+(-0.015)*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-(-0.015)*cos(ori_index*45*pi/180)]
        if LineString([[soft_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)], soft_fingertip_position, [soft_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]]).within(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.005, cap_style=2, join_style=2))==False:
            continue
        soft_fingertip_position2 = [hori_pose_Go_stone_set[Go_index][0]+(-0.018)*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-(-0.018)*cos(ori_index*45*pi/180)]
        if LineString([[soft_fingertip_position2[0]+0.01*cos(gripper_orientation*pi/180), soft_fingertip_position2[1]+0.01*sin(gripper_orientation*pi/180)], soft_fingertip_position2, [soft_fingertip_position2[0]-0.01*cos(gripper_orientation*pi/180), soft_fingertip_position2[1]-0.01*sin(gripper_orientation*pi/180)]]).within(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.005, cap_style=2, join_style=2))==False:
            continue
        fingertip_extend_position = [hori_pose_Go_stone_set[Go_index][0]+0.15*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[Go_index][1]-0.15*cos(ori_index*45*pi/180)]
        fingertip_extend_position1 = [fingertip_extend_position[0]+0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]+0.01*sin(gripper_orientation*pi/180)]
        fingertip_extend_position2 = [fingertip_extend_position[0]-0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]-0.01*sin(gripper_orientation*pi/180)]
        intersect_at_bowl = list(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(0, cap_style=2, join_style=2).boundary.intersection(LineString([hard_fingertip_position, fingertip_extend_position])).coords[:][0])
        intersect_at_bowl1 = list(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(0, cap_style=2, join_style=2).boundary.intersection(LineString([hard_fingertip_position1, fingertip_extend_position1])).coords[:][0])
        intersect_at_bowl2 = list(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(0, cap_style=2, join_style=2).boundary.intersection(LineString([hard_fingertip_position2, fingertip_extend_position2])).coords[:][0])
        if Point(hard_fingertip_position).distance(Point(intersect_at_bowl))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.0365): # need to be changed
            continue
        if Point(hard_fingertip_position1).distance(Point(intersect_at_bowl1))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.0365): # need to be changed
            continue
        if Point(hard_fingertip_position2).distance(Point(intersect_at_bowl2))*sqrt(3)<0.07-(hori_pose_Go_stone_set[Go_index][2]-0.0365): # need to be changed
            continue
        return False
    return True  

In [14]:
scoop_collision_check(0)

NameError: global name 'Go_stone_dir_sorted' is not defined

In [14]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

# first shorten, next translate, finally close
# l0=0.0125 l1=0.1 l2l=0.006 l2r=0.015 flength_l = 0.072   theta=60    joint_value = 78.88, -112.01, -106.91, -61.26, 133.28, -15.20
def pos_flength_adjust_left_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    translation_dir_dis[0] = translation_dir_dis[0]+(2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/sin(theta)-(next_right_fingertip_pos_after_trans_w[0]-right_finger_shorten_distance*cos(theta))
    return translation_dir_dis, right_finger_shorten_distance

def pos_flength_adjust_right_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), cos(theta), -current_right_fingertip_pos_g[0]*sin(theta)-current_right_fingertip_pos_g[1]*cos(theta)], [-cos(theta), sin(theta), current_right_fingertip_pos_g[0]*cos(theta)-current_right_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_left_fingertip_pos_g[0]], [current_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    #print current_left_fingertip_pos_w
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    #print next_left_fingertip_pos_w
    translation_dir_dis = [-next_right_fingertip_pos_w[0], -next_right_fingertip_pos_w[1]]
    #print translation_dir_dis
    next_left_fingertip_pos_after_trans_w = [next_left_fingertip_pos_w[0]+translation_dir_dis[0], next_left_fingertip_pos_w[1]+translation_dir_dis[1]]
    #print next_left_fingertip_pos_after_trans_w[1]
    left_finger_shorten_distance=abs(next_left_fingertip_pos_after_trans_w[1])/sin(theta)
    return translation_dir_dis, left_finger_shorten_distance

def scoop_execute(Go_index):
    global hard_fingertip_position, gripper_orientation
    softtip()
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
    servo_pub.publish(0)
    rob.movel_tool((0,0,0,0,0,-gripper_orientation*pi/180),acc=0.1, vel=0.2)
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = -0.021 # need change
    tcp2fingertip.pos.z = 0.342 # need change
    world2tcp = rob.get_pose()
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hard_fingertip_position[0]-world2fingertip.pos.x,hard_fingertip_position[1]-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
    rob.set_tcp((-0.021, 0, 0.342, 0, 0, 0))
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)    
    rob.translate((0, 0, -0.07), acc=0.1, vel=0.3, wait=True)
    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    torque_memory=[]
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        torque_memory.append(wrench.wrench.torque.y)
        if len(torque_memory)<9:
            if wrench.wrench.torque.y>ini_wrench+0.15:
                rob.stopl()
                break
        else:
            if wrench.wrench.torque.y>min(torque_memory[0:9])+0.15:
                rob.stopl()
                break

    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    torque_memory=[]
    for current_finger_length in np.arange(0,100,1):
        servo_pub.publish(current_finger_length)
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        torque_memory.append(wrench.wrench.torque.y)
        if len(torque_memory)<9:
            if wrench.wrench.torque.y>ini_wrench+0.15:
                rob.stopl()
                break
        else:
            if wrench.wrench.torque.y>min(torque_memory[0:9])+0.15:
                rob.stopl()
                break
    angle_step=0.25
    for angle in np.arange(5,-4,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(160.0/0.025),0))
        servo_pub.publish(int(current_fingerlength_int))
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
            rospy.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=(-4)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=True)
    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)

In [15]:
def scoop_execute2(Go_index):
    global hard_fingertip_position, gripper_orientation
    softtip()
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
    servo_pub.publish(0)
    rob.movel_tool((0,0,0,0,0,-gripper_orientation*pi/180),acc=0.1, vel=0.2)
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = -0.021 # need change
    tcp2fingertip.pos.z = 0.342 # need change
    world2tcp = rob.get_pose()
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hard_fingertip_position[0]-world2fingertip.pos.x,hard_fingertip_position[1]-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
    rob.set_tcp((-0.021, 0, 0.342, 0, 0, 0))
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    rob.translate((0, 0, hori_pose_Go_stone_set[Go_index][2]-world2fingertip.pos.z+0.004), acc=0.02, vel=0.05, wait=True)

    for current_finger_length in np.arange(0,85,1):
        servo_pub.publish(current_finger_length)
        rospy.sleep(0.02) #need change
    current_fingerlength_int=85
    angle_step=0.25
    for angle in np.arange(5,-4,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(160.0/0.025),0))
        servo_pub.publish(int(current_fingerlength_int))
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
            rospy.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=(-4)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=True)
    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)

In [16]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    hardtip()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[Go_index]+0.0095, speed=0.1, force=10, block=True)
    servo_pub.publish(10)    
    pos_x = rob.get_pose().pos.x+0.007
    pos_y = rob.get_pose().pos.y-0.001
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x,ver_pose_Go_stone_set[Go_index][1]-pos_y,0), acc=0.3, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.3, vel=0.8)
    rob.translate_tool((0.004,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,(ver_pose_Go_stone_set[Go_index][2]-(rob.get_pose().pos.z-0.32210))*0.7), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,(ver_pose_Go_stone_set[Go_index][2]-(rob.get_pose().pos.z-0.32210))*0.3-0.003), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)

In [17]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing
from shapely.geometry import Polygon
 
def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, -gripper_aperture_set[Go_index]/2-0.007], [0.01, -gripper_aperture_set[Go_index]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.005, cap_style=2, join_style=2))==True:
        return False
    else:
        return True

In [1]:
stone_img_count = 0
while True:
    go_to_safe_place()
    servo_pub.publish(10)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    ver_pose_Go_stone_set = []
    gripper_direction_set = []
    gripper_aperture_set = []
    is_send_stone_img = 1
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(yaw_pitch_normal_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(gripper_aperture_set):
            break
        rospy.sleep(0.1)   
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    print ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    if hori_pose_Go_stone_set==[] and ver_pose_Go_stone_set == []:
        continue
    priority_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            priority_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            priority_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            for index_ver in range(len(ver_pose_Go_stone_set)):
                if hori_pose_Go_stone_set[index_hori][2]<=ver_pose_Go_stone_set[index_ver][2]-0.002 and ('v', index_ver) not in priority_list:
                    priority_list.append(('v', index_ver))
                v_num = 0
                for element in priority_list:
                    if element[0]=='v':
                        v_num+=1
                if (v_num == len(ver_pose_Go_stone_set) or hori_pose_Go_stone_set[index_hori][2]>ver_pose_Go_stone_set[index_ver][2]-0.002) and ('h', index_hori) not in priority_list:
                    priority_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            if ('v', index_ver) not in priority_list:
                priority_list.append(('v', index_ver))
    print priority_list
    for index in priority_list:
        if index[0]=='v':
            continue
            if direct_pick_collision_check(index[1])==False and ver_pose_Go_stone_set[index[1]][2]<0.085:
                direct_pick_execute(index[1])
                go_to_end_place()
                rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                softtip()
                rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                break
        elif index[0]=='h':
            if hori_pose_Go_stone_set[index[1]][2]>0.085 or hori_pose_Go_stone_set[index[1]][2]<0:
                continue
            if hori_pose_Go_stone_set[index[1]][2]>0.06:
                if Point([hori_pose_Go_stone_set[index[1]][0], hori_pose_Go_stone_set[index[1]][1]]).within(Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.006, cap_style=2, join_style=2))==True:
                    if poke_grasp_execute(index[1])==True:
                        go_to_end_place()
                        rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                        Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                        softtip()
                        rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                        break                
            elif hori_pose_Go_stone_set[index[1]][2]>0.04:
                if scoop_collision_check(index[1])==False:
                    scoop_execute2(index[1]) 
                    go_to_end_place()
                    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                    rospy.sleep(0.5)
                    servo_pub.publish(180)
                    Robotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)
                    rospy.sleep(1)
                    servo_pub.publish(10)
                    rospy.sleep(1)
                    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                    break
            else:
                if scoop_collision_check(index[1])==False:
                    scoop_execute(index[1]) 
                    go_to_end_place()
                    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                    rospy.sleep(0.5)
                    servo_pub.publish(180)
                    Robotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)
                    rospy.sleep(1)
                    servo_pub.publish(10)
                    rospy.sleep(1)
                    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                    break
    else:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001
        if ver_pose_Go_stone_set == []:
            min_height = np.min(np.array(hori_pose_Go_stone_set)[:,2])
        elif hori_pose_Go_stone_set == []:
            min_height = np.min(np.array(ver_pose_Go_stone_set)[:,2])
        else:
            min_height = min(np.min(np.array(hori_pose_Go_stone_set)[:,2]), np.min(np.array(ver_pose_Go_stone_set)[:,2]))          
        if min_height>0.04:
            hardtip()
            world2tcp = rob.get_pose()
            world2tcp.pos.x+=0.004
            world2tcp.pos.y-=0.002
            tcp2fingertip=m3d.Transform()
            tcp2fingertip.pos.x = 0
            tcp2fingertip.pos.y = 0  
            tcp2fingertip.pos.z = 0.354
            world2fingertip=(world2tcp*tcp2fingertip)  
            rob.translate((0.05-pos_x,0.675-pos_y,0), acc=0.1, vel=0.4, wait=True)
            descend_distance = min_height-world2fingertip.pos.z+0.003
            rob.translate((0,0,descend_distance), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.01,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0.03,0,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((-0.03,0.04,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((-0.03,-0.04,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0.03,0,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0,0,-descend_distance), acc=0.1, vel=0.4, wait=True)  
        else:
            if hori_pose_Go_stone_set==[]:
                hardtip()
                servo_pub.publish(10)
                rob.translate((0.05-pos_x,0.675-pos_y,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0,-0.07), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,-0.02,0), acc=0.1, vel=0.4, wait=True) 
                rob.translate((0.045,0,0), acc=0.1, vel=0.4, wait=True) 
                rob.translate((-0.045,0.08,0), acc=0.1, vel=0.4, wait=True) 
                rob.translate((-0.045,-0.08,0), acc=0.1, vel=0.4, wait=True) 
                rob.translate((0.045,0,0), acc=0.1, vel=0.4, wait=True) 
                rob.translate((0,0,0.07), acc=0.1, vel=0.4, wait=True)
            else:
                softtip()
                servo_pub.publish(100)
                if len(hori_pose_Go_stone_set)<=2:
                    sweep_index_set = range(len(hori_pose_Go_stone_set))
                else:
                    sweep_index_set = random.sample(range(len(hori_pose_Go_stone_set)), 2)
                for hori_index in sweep_index_set:
                    print hori_index
                    alpha = atan2(0.665-hori_pose_Go_stone_set[hori_index][1], 0.05-hori_pose_Go_stone_set[hori_index][0])
                    if alpha > pi/2 and alpha<=pi:
                        rotate_angle = alpha-3*pi/2
                    else:
                        rotate_angle = alpha+pi/2
                    rob.movel_tool((0,0,0,0,0,-rotate_angle), acc=0.1, vel=0.2, wait=True) 
                    world2tcp = rob.get_pose()
                    world2tcp.pos.x+=0.004
                    world2tcp.pos.y-=0.002
                    tcp2fingertip=m3d.Transform()
                    tcp2fingertip.pos.x = 0
                    tcp2fingertip.pos.y = 0  
                    tcp2fingertip.pos.z = 0.354
                    world2fingertip=(world2tcp*tcp2fingertip)
                    if Point([hori_pose_Go_stone_set[hori_index][0], hori_pose_Go_stone_set[hori_index][1]]).within(Polygon([[0.07+0.05,0.635], [0.05,0.155+0.6], [-0.07+0.05,0.635]]))==False:
                        Point_prime = LinearRing([[0.07+0.05,0.635], [0.05,0.155+0.6], [-0.07+0.05,0.635]]).interpolate(LinearRing([[0.07+0.05,0.635], [0.05,0.155+0.6], [-0.07+0.05,0.635]]).project(Point([hori_pose_Go_stone_set[hori_index][0], hori_pose_Go_stone_set[hori_index][1]])))
                        print Point_prime.coords[:]
                        coordinate_x = Point_prime.coords[:][0][0]
                        coordinate_y = Point_prime.coords[:][0][1]
                    else:
                        coordinate_x = hori_pose_Go_stone_set[hori_index][0]
                        coordinate_y = hori_pose_Go_stone_set[hori_index][1]    
                    
                    rob.translate((coordinate_x-world2fingertip.pos.x,coordinate_y-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
                    rob.movel_tool((0.002,0,0,0,0,0), acc=0.1, vel=0.2, wait=True) 
                    move_ref_tcp((0,0,0,0,20*pi/180,0), (0.002, 0, 0.354, 0, 0, 0), acc=0.3, vel=0.8, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.1, vel=0.3, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
                    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                    ini_wrench = wrench.wrench.force.z
                    torque_memory=[]
                    while True:
                        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                        torque_memory.append(wrench.wrench.force.z)
                        if len(torque_memory)<9:
                            if wrench.wrench.force.z<ini_wrench-5:
                                rob.stopl()
                                break
                        else:
                            if wrench.wrench.force.z<max(torque_memory[0:9])-5:
                                rob.stopl()
                                break 
                    point_to_bowl_center_length=sqrt((coordinate_x-0.05)**2+(coordinate_y-0.665)**2)
                    rob.translate((point_to_bowl_center_length*0.8*cos(alpha), point_to_bowl_center_length*0.8*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    #hardtip()
                    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
                    rob.movej((radians(99.10),radians(-91.67), radians(-107.60), radians(-70.72), radians(89.13), radians(-80.76)), 0.3, 0.7)




    

NameError: name 'go_to_safe_place' is not defined

In [40]:
((0.075-0.07331093786463667)+(0.05-0.04895368329465694)+(0.025-0.022842304328039287)+(0.025-0.024034588402665067))/4.0

0.0014646215275005078

In [41]:
((0.675-0.6745454043861371)+(0.675-0.6728774495939114)+(0.7-0.6973225984467233)+(0.65-0.6470066321920008))/4.0

0.002061978845306872

In [21]:
softtip()

In [31]:
poke_grasp_execute(0)

[[-2.6201582138180495, 0.8652281491096858, (nan, nan, nan)], [-0.5954098754787327, 0.5467888408892477, (-0.70385285157761, 0.7103383061068284, -0.003264077717916657)], [-1.7731179551732001, 0.9475969968608307, (nan, nan, nan)], [2.5766882515590166, 0.6365082157879511, (-0.17460723996886035, 0.8738680628139778, -0.4200974798129315)]]


False

In [38]:
Robotiq.goto(robotiq_client, pos=0.015, speed=0.1, force=10, block=True)
print find_alpha_from_distance(0.01, 0.0125, 0.1, 0.006, 0.02)

3.15285880405


In [7]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.1, force=10, block=True)

In [ ]:
go_to_home()

In [27]:
rob.translate((0, 0, -0.01), acc=0.1, vel=0.2, wait=False)

In [56]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=False)

In [13]:
#rob.movel_tool((0,0,0,0,0,pi),acc=0.1, vel=0.2)
pos_x = rob.get_pose().pos.x
pos_y = rob.get_pose().pos.y
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.4-pos_y,0), acc=0.1, vel=0.4, wait=True)
print rob.get_pose()

<Transform:
<Orientation: 
array([[-0.01289458,  0.99989471,  0.0066557 ],
       [ 0.99991571,  0.01288417,  0.00160434],
       [ 0.00151842,  0.00667583, -0.99997656]])>
<Vector: (0.05006, 0.39995, 0.22845)>
>


In [14]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.5, force=10, block=True)

In [20]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)

rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)

KeyboardInterrupt: 

In [25]:
go_to_safe_place()
softtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01305109,  0.99989524,  0.00625989],
       [ 0.99991361,  0.01304105,  0.00164169],
       [ 0.00155988,  0.00628078, -0.99997906]])>
<Vector: (0.04293, 0.85097, 0.45825)>
>

In [9]:
go_to_safe_place()
hardtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

<Transform:
<Orientation: 
array([[-0.01308933,  0.99989511,  0.00619996],
       [ 0.99991303,  0.01307907,  0.00169334],
       [ 0.00161207,  0.00622159, -0.99997935]])>
<Vector: (0.04282, 0.80750, 0.45825)>
>

In [5]:
from sympy import symbols, solve
import time
time0 = time.time()
x = symbols('x')
f1 = 2 * x + 6 - 7 * x + 9
f2 = 2 * (4 * x + 2) - 5 * (x - 1)
print solve([f1 > 0, f2 >= 0])
print time.time()-time0

(-3 <= x) & (x < 3)
0.0127120018005


In [22]:
Robotiq.goto(robotiq_client, pos=0.75*0.012+0.0095, speed=0.1, force=10, block=True)
softtip()

In [32]:
for i in range(1,24):
    servo_pub.publish(round(150-72/11*i))
    rob.translate_tool((0,0,0.001), acc=0.1, vel=0.2, wait=True) 

In [37]:
print rob.get_pose()

<Transform:
<Orientation: 
array([[-0.01294484,  0.99989614,  0.00633607],
       [ 0.99991511,  0.01293516,  0.00156607],
       [ 0.00148395,  0.0063558 , -0.9999787 ]])>
<Vector: (0.04309, 0.85098, 0.02115)>
>


In [20]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.1, force=10, block=True)

In [63]:
hardtip()

In [33]:
rob.set_tcp((0, 0.003, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,0,-pi), acc=0.1, vel=0.2, wait=True)

In [27]:
go_to_safe_place()
rob.set_tcp((-0.002, 0, 0.354, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,0,0*pi/180), acc=0.1, vel=0.2, wait=True)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
#rob.movel_tool((0,0,0,0,-20*pi/180,0), acc=0.1, vel=0.2, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
softtip()
#go_to_safe_place()
servo_pub.publish(100)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.002
tcp2fingertip.pos.y = 0  
tcp2fingertip.pos.z = 0.354
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y-=0.002
world2tcp.orient=np.array([[0,1,0],[1,0,0],[0,0,-1]])
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01288596,  0.9998975 ,  0.0062404 ],
       [ 0.99991523,  0.01287405,  0.00194468],
       [ 0.00186414,  0.00626493, -0.99997864]])>
<Vector: (0.04603, 0.85399, 0.39044)>
>

In [29]:
rob.set_tcp((-0.002, 0, 0.354, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.1, vel=0.2, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.1, vel=0.2, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [71]:
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = 0.002
tcp2fingertip.pos.y = 0   
tcp2fingertip.pos.z = 0.356
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.008
world2tcp.pos.y-=0.001
world2fingertip=(world2tcp*tcp2fingertip)
print world2tcp, world2fingertip, tcp2fingertip

<Transform:
<Orientation: 
array([[ 0.99989078,  0.01345517,  0.00611427],
       [ 0.01344501, -0.99990817,  0.00170076],
       [ 0.0061366 , -0.00161837, -0.99997986]])>
<Vector: (0.04980, 0.79797, 0.39029)>
> <Transform:
<Orientation: 
array([[ 0.99989078,  0.01345517,  0.00611427],
       [ 0.01344501, -0.99990817,  0.00170076],
       [ 0.0061366 , -0.00161837, -0.99997986]])>
<Vector: (0.05398, 0.79860, 0.03431)>
> <Transform:
<Orientation: 
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])>
<Vector: (0.00200, 0.00000, 0.35600)>
>


In [73]:
print rob.get_pose()

<Transform:
<Orientation: 
array([[-0.01293961,  0.99989677,  0.00624669],
       [ 0.9999151 ,  0.01292975,  0.00161633],
       [ 0.0015354 ,  0.00626707, -0.99997918]])>
<Vector: (0.04318, 0.54331, 0.45826)>
>


In [28]:
go_to_safe_place()

In [33]:
Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
servo_pub.publish(170)
softtip()
rospy.sleep(1)
'''
Robotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)
rospy.sleep(1)
servo_pub.publish(10)
rospy.sleep(1)
'''

'\nRobotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)\nrospy.sleep(1)\nservo_pub.publish(10)\nrospy.sleep(1)\n'

In [65]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.5, force=10, block=True)
rospy.sleep(1)

In [32]:
servo_pub.publish(10)

In [53]:
rob.set_tcp((-0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,30*pi/180,0), acc=0.1, vel=0.2, wait=True)

6.79492950439e-05


In [26]:
go_to_safe_place()
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rob.translate((0.05-pos_x+0.02,0.69-pos_y-0.02,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((-0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,-0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True)  

<Transform:
<Orientation: 
array([[-0.01328703,  0.99989138,  0.00637776],
       [ 0.99991118,  0.0132801 ,  0.00112777],
       [ 0.00104295,  0.00639218, -0.99997903]])>
<Vector: (0.06282, 0.67096, 0.45798)>
>

In [41]:
        hardtip()
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


RobotException: Robot stopped

In [26]:
go_to_safe_place()

In [27]:
go_to_home()

In [44]:
print np.max(np.array(hori_pose_Go_stone_set)[:,2])

0.035883569954526295


IndexError: too many indices for array

In [22]:
Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [12]:
servo_pub = rospy.Publisher('servo1', UInt16, queue_size=10)
for current_finger_length in np.arange(0,135,1):
    servo_pub.publish(current_finger_length)
    rospy.sleep(0.02) #need change

In [28]:
rob.set_tcp((-0.021, 0, 0.342, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [34]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
servo_pub.publish(0)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.021 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0
world2tcp.pos.y-=0
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)


hard_fingertip_position [0.05393207309979217, 0.7060688410464139]


<Transform:
<Orientation: 
array([[-0.01286919,  0.99989759,  0.00626029],
       [ 0.99991591,  0.01285892,  0.00167886],
       [ 0.00159819,  0.00628137, -0.99997899]])>
<Vector: (0.04747, 0.87057, 0.39035)>
>

KeyboardInterrupt: 

In [37]:
            Robotiq.goto(robotiq_client, pos=(-5-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)


In [21]:
go_to_end_place()

In [15]:
from shapely.geometry import Polygon

print Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(-0.01, cap_style=2, join_style=2).boundary

#print Polygon([[0.07794+0.05,0.63], [0.05,0.735], [-0.07794+0.05,0.63]]).buffer(0, cap_style=2, join_style=2).centroid

#Polygon([[0.07+0.05,0.635], [0.05,0.155+0.6], [-0.07+0.05,0.635]])

LINESTRING (0.1080632756600323 0.64, -0.008063275660032312 0.64, 0.05 0.7182222728291429, 0.1080632756600323 0.64)


In [43]:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001    
        if True:
            hardtip()
            servo_pub.publish(10)
            world2tcp = rob.get_pose()
            world2tcp.pos.x+=0.004
            world2tcp.pos.y-=0.002
            tcp2fingertip=m3d.Transform()
            tcp2fingertip.pos.x = 0
            tcp2fingertip.pos.y = 0  
            tcp2fingertip.pos.z = 0.354
            world2fingertip=(world2tcp*tcp2fingertip)  
            rob.translate((0.05-pos_x,0.675-pos_y,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.02,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0.03,0,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((-0.03,0.05,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((-0.05,-0.05,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0.05,0,0), acc=0.1, vel=0.4, wait=True) 
            rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True)


In [49]:
rob.translate((-0.045,0.085,0), acc=0.1, vel=0.4, wait=True) 

<Transform:
<Orientation: 
array([[-1.30792334e-02,  9.99892893e-01,  6.56783956e-03],
       [ 9.99914422e-01,  1.30808411e-02, -2.01897809e-04],
       [-2.87789050e-04,  6.56463683e-03, -9.99978411e-01]])>
<Vector: (0.04323, 0.75587, 0.38771)>
>

In [50]:
                    go_to_end_place()
                    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                    rospy.sleep(0.5)
                    servo_pub.publish(180)
                    Robotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)
                    rospy.sleep(1)
                    servo_pub.publish(10)
                    rospy.sleep(1)
                    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

<Transform:
<Orientation: 
array([[-0.01311306,  0.99989542,  0.00609865],
       [ 0.99991329,  0.01310545,  0.00128495],
       [ 0.00120489,  0.00611497, -0.99998058]])>
<Vector: (0.04315, 0.40480, 0.45822)>
>